Make a table with functions and  with all the steps and flow chart
installation instructions

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import epanet_toolkit as epa
import msx_toolkit as msx
import pandas as pd
import wntr
import mf_msx_toolkit as mf

In [3]:

#Close any files that were already open
epa.ENclose()
msx.MSXclose()

#open the inp file

inp_file = 'Net1.inp'
epa.ENopen(inp_file,'report.rpt')
#open msx file
msx.MSXopen('Net1-NH2CL_JV_TOC.msx')

In [4]:
#Number of days to run model
days=14

#0 for duration
epa.ENsettimeparam(0, int(days*24*3600))

#Solve the hydraulics of the model
msx.MSXsolveH()   



### MSXRunQual

Runs an EPANET MSX simulation and saves simulation results to a dictionary, similar to how results are stored when using WNTR to run a hydraulic simulation
<br>

**Function Inputs**
<br>

<code>links</code>: A list of links in the model for which the quality results should be extracted. By default, results from all links are extracted. An empty list signifies that no results from links will be extracted.
<br>
<code>nodes</code>: A list of nodes in the model for which the quality results should be extracted. By default, results from all nodes are extracted. An empty list signifies that no results from nodes will be extracted.
<br>
<code>species</code>: A list of species in the model for which the quality results should be extracted. By default, results from all species are extracted. 
<br>
<code>by_species</code>:"yes" or "no". If "yes", the results dictionary will be structured by having the results of a dataframe for each species, and the columns of the dataframe will be each node or link. If "no", the results dictionary will be strudcutred by having a dataframe for each node/link, and each column will be a different species.
<br>
<code>t_start</code>:The time in the simulation in which simulation results should be begin to be extracted. For example, if a model is run for 14 days, but only the final 24-hour simulation results are needed, then the parameter t_start should be 13. This parameter only affects the output of results and not any parameters of the epanetmsx simulation.
<br>
<code>bin</code>:"yes" or "no". If "yes", the model will be run by saving all results to a temporary binary file, which is read back into python. If "no", model results will be extracted directly from the model after each timestep. The resulting model simulation results are identical- this affects the method by which results are extracted and may affect the time required to complete a simulation. The code in the "yes" case was developed by Jon Buckhardt.


In [5]:
#Run the reaction model
print('Running Sim')
#In this use, the function inputs "nodes" and "bin" are using the default entrie, thus not included in the 
#function call below
my_results=mf.MSXRunQual(links=[],species=['cNH2CL','T'],by_species='yes',t_start=13)
print('Ending Sim')

Running Sim
Ending Sim


### GetNodeNameList

Returns a list of all nodes in the model

There are no inputs to this function

In [6]:
nodes=mf.GetNodeNameList()
print(nodes)

['10', '11', '12', '13', '21', '22', '23', '31', '32', '9', '2']


### GetLinkNameList

Returns a list of all links in the model

There are no inputs to this function

In [7]:
links=mf.GetLinkNameList()
print(links)

['10', '11', '12', '21', '22', '31', '110', '111', '112', '113', '121', '122', '9']


### GetSpeciesNameList

Returns a list of all species in the model

There are no inputs to this function

In [8]:
species=mf.GetSpeciesNameList()
print(species)

['HOCL', 'TOTCL', 'NH3', 'TOTNH', 'NH2CL', 'NHCL2', 'NCL3', 'I', 'OCL', 'NH4', 'TOTCO', 'H', 'OH', 'CO3', 'HCO3', 'H2CO3', 'cNH3', 'cNH2CL', 'AGE', 'DOC1', 'DOC2', 'T']


### GetConstantNameList

Returns a list of all constants in the model

There are no inputs to this function

In [9]:
constants=mf.GetConstantNameList()
print(constants)

['k1', 'k2', 'k3', 'k4', 'k6', 'k7', 'k8', 'k9', 'k10', 'k12', 'k13', 'k14', 'k11CO3', 'k11OCL', 'k11OH', 'kDOC1', 'kDOC2', 'AC1', 'AC2', 'AC3', 'KNH4', 'KHOCL', 'KH2CO3', 'KHCO3']


### GetConstants

GetConstants(con_get)

Returns an array containing the values of specified constants in the model

<code>con_get</code>: A list containing the IDs of the constants for which the values should be extracted

The sequence of values in the output array correspond to the sequence of constants in the <code>con_get</code> function input

In [10]:
con_vals=mf.GetConstants(['k1','k3','kDOC2'])
print(con_vals)

[[4.2e+06]
 [2.8e+02]
 [1.8e+02]]


### GetInitialConcentration

GetInitialConcentration(node,species)

Returns the values of initial species concentrations

<code>node</code>: a string with the node ID of interest

<code>species</code>: A list with the species of interest

The sequence of values in the output array correspond to the sequence of species in the <code>species</code> function input


In [12]:
init_vals=mf.GetInitialConcentration('9',['TOTNH','TOTCO'])
print(init_vals)

[[9.e-06]
 [4.e-03]]


### SetConstants

Sets constants within the MSX model to a desired value

SetConstants(con_get,given_constants)

<code>con_get</code>: A list containg the constants in the MSX model to be changed
<br>
<code>given_constants</code>: An array containing the desired values of the constants

The sequence of the values in the <code>given_constants</code> array corresponds to the sequence of constants in the list <code>con_get</code>

In [15]:
#Get the values of the K1 and K2 constants
cons=mf.GetConstants(['k1','k2'])
print('The original values of k1 and k2 are: ' + str(cons[0]) + ' and ' +str(cons[1]))

#Increase values of k1 and k2 by a factor of 1.5
mf.SetConstants(['k1','k2'],cons*1.5)

#Extract the modified values of the constants to show that they were changed
cons_mod=mf.GetConstants(['k1','k2'])
print('The modified values of k1 and k2 are: ' + str(cons_mod[0]) + ' and ' +str(cons_mod[1]))



The original values of k1 and k2 are: [4200000.] and [2.1e-05]
The modified values of k1 and k2 are: [6300000.] and [3.15e-05]


### SetInitialConcentration

SetInitialConcentration(node, species, init_val)

Sets the inital concentration of species at a specified node

<code>node</code>: A string with the node ID of interest<br>
<code>species</code>: A list of the species to be changed<br>
<code>init_val</code>: An array of the values of the species to be changed


The sequence of the values in the <code>init_val</code> array corresponds to the sequence of species in the list <code>species</code>

In [17]:
#Get the initial concentration of TOTCO and TOTNH species at Node 9
init_vals=mf.GetInitialConcentration('9',['TOTNH','TOTCO'])
print('The original concentrations of TOTNH and TOTCO are: ' + str(init_vals[0]) + ' and ' +str(init_vals[1]))

#Increase initial concentrations of TOTNH and TOTCO at node 9 by a factor of 2
mf.SetInitialConcentration('9',['TOTNH','TOTCO'],init_vals*2)


#Get the initial concentration of TOTCO and TOTNH species at Node 9
mod_vals=mf.GetInitialConcentration('9',['TOTNH','TOTCO'])
print('The modified concentrations of TOTNH and TOTCO are: ' + str(mod_vals[0]) + ' and ' +str(mod_vals[1]))



The original concentrations of TOTNH and TOTCO are: [9.e-06] and [0.004]
The modified concentrations of TOTNH and TOTCO are: [1.8e-05] and [0.008]


### SetGlobalInitialConcentration

SetGlobalInitialConcentration(species,init_val)

Sets the initial concentration of a species at all links and nodes in the model

<code>species</code>:A list of the species to be changed<br>
<code>init_val</code>: An array of the values of the species


In [18]:
#Change the pH to 7.25
mf.SetGlobalInitialConcentration(['H'],np.array([10**-7.25]))